In [1]:
import cv2, torch
import numpy as np
import pandas as pd
import pytorch_lightning as pl
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
from PIL import Image

In [2]:
class EyeDataset(Dataset):
    def __init__(self, dataset_desc: pd.DataFrame, transform: transforms):
        self.h_offset_scale = 0.05
        self.v_offset_scale = 0.05

        self.__dataset_desc = dataset_desc
        self.__transform = transform
        self.__img_file_name = ''
        self.__eye_position = list()
        self.__img = np.array([])
        self.__eye_img = np.array([])

    def __len__(self):
        return self.__dataset_desc.shape[0]

    def __getitem__(self, idx):
        self.__img_file_name = self.__dataset_desc.loc[idx, 'file_name']
        self.__img = cv2.imread(self.__img_file_name)

        self.__eye_position = [self.__dataset_desc.loc[idx, 'left_eye_pt1_x_pos'],
                               self.__dataset_desc.loc[idx, 'left_eye_pt1_y_pos'],
                               self.__dataset_desc.loc[idx, 'left_eye_pt2_x_pos'],
                               self.__dataset_desc.loc[idx, 'left_eye_pt2_y_pos']]

        self.__eye_position[0] = int(self.__eye_position[0] - (self.__eye_position[0] * self.h_offset_scale))
        self.__eye_position[1] = int(self.__eye_position[1] - (self.__eye_position[1] * self.v_offset_scale))
        self.__eye_position[2] = int(self.__eye_position[2] + (self.__eye_position[2] * self.h_offset_scale))
        self.__eye_position[3] = int(self.__eye_position[3] + (self.__eye_position[3] * self.v_offset_scale))

        self.__eye_img = self.__img[self.__eye_position[1]:self.__eye_position[3],
                         self.__eye_position[0]:self.__eye_position[2]]

        if self.__transform:
            self.__eye_img = Image.fromarray(self.__eye_img)
            self.__eye_img = self.__transform(self.__eye_img)
        else:
            self.__eye_img = Image.fromarray(self.__eye_img)

        return self.__eye_img, int(self.__dataset_desc.loc[idx, 'left_eye_opened'])


class EyeDataModule(pl.LightningDataModule):
    def __init__(self, train_dataset_desc: pd.DataFrame, val_dataset_desc: pd.DataFrame, batch_size: int = 16,
                 n_workers: int = 8):
        super().__init__()
        self.__train_dataset_desc = train_dataset_desc
        self.__val_dataset_desc = val_dataset_desc
        self.batch_size = batch_size
        self.num_workers = n_workers
        self.__transform = transforms.Compose([transforms.Resize((64, 64)),
                                               transforms.ToTensor(),
                                               transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                                    std=[0.229, 0.224, 0.225])])

    def setup(self, stage=None):
        self.__train_dataset = EyeDataset(self.__train_dataset_desc, transform=self.__transform)
        self.__val_dataset = EyeDataset(self.__val_dataset_desc, transform=self.__transform)

    def train_dataloader(self):
        return DataLoader(self.__train_dataset, batch_size=self.batch_size, shuffle=True, num_workers=self.num_workers,
                          persistent_workers=True)

    def val_dataloader(self):
        return DataLoader(self.__val_dataset, batch_size=self.batch_size, shuffle=False, num_workers=self.num_workers,
                          persistent_workers=True)
    
    def predict_dataloader(self):
        return DataLoader(self.__val_dataset, batch_size=self.batch_size, shuffle=False, num_workers=self.num_workers)

In [3]:
class Model(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.__model = models.resnet18(weights=None)
        self.__model.conv1 = torch.nn.Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        self.__model.fc = torch.nn.Linear(in_features=self.__model.fc.in_features, out_features=2)
        self.__criterion = torch.nn.CrossEntropyLoss()

    def forward(self, x):
        return self.__model(x)

    def training_step(self, batch, batch_idx):
        img, label = batch
        label = label.float()
        pred = self.forward(img)
        loss = self.__criterion(pred, label)
        self.log('train_loss', loss)

        return loss

    def validation_step(self, batch, batch_idx):
        img, label = batch
        label = label.float()
        pred = self.forward(img)
        loss = self.__criterion(pred, label)
        self.log('val_loss', loss)

        return loss
    
    def predict_step(self, batch, batch_idx):
        img, label = batch
        pred = self.forward(img)
        
        return pred
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.__model.parameters(), lr=0.001)

        return optimizer

In [4]:
data = EyeDataModule(train_dataset_desc=pd.read_csv('train_eye_shape_dataset.csv'), val_dataset_desc=pd.read_csv('test_eye_shape_dataset.csv'), batch_size=1, n_workers=0)

In [6]:
model = Model.load_from_checkpoint('epoch=5-step=14832.ckpt')

In [7]:
trainer = pl.Trainer()
pred = trainer.predict(model, datamodule=data)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
C:\workspace\drowsy_detection\venv\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Predicting: |          | 0/? [00:00<?, ?it/s]

In [8]:
pred

[tensor([[-1.2154,  1.0171]]),
 tensor([[-1.2124,  1.0127]]),
 tensor([[-1.2123,  1.0129]]),
 tensor([[-1.2222,  1.0256]]),
 tensor([[-1.2159,  1.0185]]),
 tensor([[-1.2125,  1.0128]]),
 tensor([[-1.2197,  1.0239]]),
 tensor([[-1.2145,  1.0148]]),
 tensor([[-1.2124,  1.0128]]),
 tensor([[-1.2120,  1.0105]]),
 tensor([[-1.2222,  1.0257]]),
 tensor([[-1.2138,  1.0143]]),
 tensor([[-1.2122,  1.0125]]),
 tensor([[-1.2207,  1.0238]]),
 tensor([[-1.2123,  1.0126]]),
 tensor([[-1.2125,  1.0129]]),
 tensor([[-1.2119,  1.0120]]),
 tensor([[-1.1829,  0.9910]]),
 tensor([[-1.2205,  1.0228]]),
 tensor([[-1.2240,  1.0261]]),
 tensor([[-1.2255,  1.0270]]),
 tensor([[-1.2053,  1.0097]]),
 tensor([[-1.2164,  1.0199]]),
 tensor([[-1.2126,  1.0128]]),
 tensor([[-1.2124,  1.0127]]),
 tensor([[-1.2154,  1.0176]]),
 tensor([[-1.2125,  1.0128]]),
 tensor([[-1.2121,  1.0123]]),
 tensor([[-1.2124,  1.0127]]),
 tensor([[-1.2122,  1.0124]]),
 tensor([[-1.2143,  1.0146]]),
 tensor([[-1.2123,  1.0128]]),
 tensor(

In [26]:
for pr in pred:
    print(np.argmax(pr))

tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
